# Generates watchlist for IBKR to import into

In [21]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd 

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

In [ ]:
from fast_prices import df_iv
from utils import get_ib, get_open_orders, get_pickle, quick_pf

# Get positions and orders to remove
with get_ib('SNP') as ib:
    positions = quick_pf(ib)
    ords = get_open_orders(ib)

    if len(ords) > 0:
        openords = get_open_orders(ib).symbol.to_list()
    else: 
        openords = set()
    pos_symbols = positions.symbol.to_list()
    remove = set(pos_symbols).union(((openords)))
    
# Get clean targets from unds
unds = get_pickle(ROOT / 'data' / 'snp_unds.pkl')

contracts = [c for c in unds if c.symbol not in remove]

# Get volatilities of these contracts
with get_ib('SNP') as ib:
    df = await df_iv(ib=ib, stocks=contracts)

In [27]:
# Sort df on the ratio of iv to hv in ascending order
df['ivhv'] = df.hv/df.iv
df = df.sort_values('ivhv').drop(columns='ivhv')

In [28]:
# Write to CSV
csv_txt = [f"DES:{c.symbol}"+":STK:SMART/AMEX,,,,," for c in df.contract]
with open(ROOT / 'data' / 'watchlist.csv', 'w', newline='') as file:
    for item in csv_txt:
        file.write(item.replace(':', ',') + '\n')